In [4]:
import os
import json
import joblib
import numpy as np
import pandas as pd

from pathlib import Path
from datetime import datetime

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [5]:
ROOT = os.path.abspath(os.path.join(os.getcwd(), "..")) if "EDA" in os.getcwd() or "scripts" in os.getcwd() else os.getcwd()

RAW_CSV = os.path.join(ROOT, "data", "clean_data", "clean_spotify_info.csv")

OUT_CSV = os.path.join(ROOT, "data", "clean_data", "spotify_features.csv")
META_JSON = os.path.join(ROOT, "artifacts", "feature_meta.json")

os.makedirs(os.path.dirname(OUT_CSV), exist_ok=True)
os.makedirs(os.path.dirname(META_JSON), exist_ok=True)

print("ROOT:", ROOT)
print("Input CSV:", RAW_CSV)
print("Output CSV:", OUT_CSV)
print("Meta JSON:", META_JSON)

ROOT: C:\DS\spotify-MRS
Input CSV: C:\DS\spotify-MRS\data\clean_data\clean_spotify_info.csv
Output CSV: C:\DS\spotify-MRS\data\clean_data\spotify_features.csv
Meta JSON: C:\DS\spotify-MRS\artifacts\feature_meta.json


In [6]:
columns = ["artist_name", "track_id", "track_name", "acousticness", "danceability", "duration_ms", "energy", "instrumentalness", "key", "liveness", "loudness", "mode", "speechiness", "tempo", "time_signature", "valence", "popularity"
]
df = (pd.read_csv(RAW_CSV, names = columns, header = None))
print("Rows, Cols:", df.shape)
df.head()

Rows, Cols: (130326, 17)


,artist_name,track_id,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity
0,guccihighwaters,4AG5lmD160UcX499Vn0o75,so temporary,0.7500,0.732,196174,0.468,0.000002,8,0.2850,-8.552,0,0.0569,115.060,4,0.400,46
1,Los Rugar,1G7rk1xrfsZHKDNAZiSdPY,El Olotito,0.5810,0.662,223817,0.951,0.614000,8,0.1240,-5.954,1,0.0331,104.560,4,0.930,36
2,Duexo,466uTriGm7QWSrX75F1epf,Reckless,0.0507,0.613,203571,0.616,0.000000,0,0.1880,-11.229,1,0.0555,111.982,4,0.480,15
3,JohnLMR,6q6yNb30I3VvTFsaqpP3k9,Dämonen,0.5930,0.712,268547,0.562,0.000000,8,0.0996,-8.870,1,0.2180,129.880,4,0.183,1
4,JOY.,7ypn9YucE5ULiqP4pVwv10,Smoke Too Much,0.0858,0.567,184687,0.554,0.000000,5,0.1560,-7.080,1,0.0639,100.039,4,0.216,30


**For future nlp implementation & drop unnecessary column**

In [7]:
nlp_cols = [c for c in ["artist_name", "track_name"] if c in df.columns]
df_nlp = df[nlp_cols + ["popularity"]] if nlp_cols else None

if df_nlp is not None:
    df_nlp.to_csv(os.path.join(os.path.dirname(RAW_CSV), "spotify_text_features.csv"), index = False)
df = df.drop(columns = ["track_id"], errors = "ignore")
display(df.head())

,artist_name,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity
0,guccihighwaters,so temporary,0.7500,0.732,196174,0.468,0.000002,8,0.2850,-8.552,0,0.0569,115.060,4,0.400,46
1,Los Rugar,El Olotito,0.5810,0.662,223817,0.951,0.614000,8,0.1240,-5.954,1,0.0331,104.560,4,0.930,36
2,Duexo,Reckless,0.0507,0.613,203571,0.616,0.000000,0,0.1880,-11.229,1,0.0555,111.982,4,0.480,15
3,JohnLMR,Dämonen,0.5930,0.712,268547,0.562,0.000000,8,0.0996,-8.870,1,0.2180,129.880,4,0.183,1
4,JOY.,Smoke Too Much,0.0858,0.567,184687,0.554,0.000000,5,0.1560,-7.080,1,0.0639,100.039,4,0.216,30


In [8]:
df["popularity"] = pd.to_numeric(df["popularity"], errors = "coerce")
popularity_median = df["popularity"].median()
df["popularity"] = df["popularity"].fillna(popularity_median)
popularity_threshold = 73
df["popular_flag"] = (df["popularity"] >= popularity_threshold).astype(int)
print("Target summary:")
print(df["popularity"].describe(percentiles=[0.1,0.25,0.5,0.75,0.9]).round(2))
print(f"\nPopular threshold: {popularity_threshold}")
print("Popular count:", int(df["popular_flag"].sum()),
      "| Not popular:", int((1 - df["popular_flag"]).sum()))
print("Popular ratio:", float(df["popular_flag"].sum() / (1 - df["popular_flag"]).sum()) * 100)

Target summary:
count    130326.00
mean         24.13
std          19.66
min           0.00
10%           0.00
25%           7.00
50%          21.00
75%          38.00
90%          53.00
max         100.00
Name: popularity, dtype: float64

Popular threshold: 73
Popular count: 1287 | Not popular: 129039
Popular ratio: 0.9973728872666404


**Feature Engineering**

In [9]:
engineered = []

def _num(s):
    return pd.to_numeric(s, errors = "coerce")

core = [c for c in ["acousticness", "danceability", "duration_ms", "energy", "instrumentalness", "liveness", "loudness", "speechiness", "tempo", "valence"
] if c in df.columns]

if {"tempo", "duration_ms"}.issubset(df.columns):
    df["total_beats"] = (_num(df["tempo"]) * _num(df["duration_ms"]) / 60000)
    df["total_beats"] = df["total_beats"].replace([np.inf, -np.inf], np.nan).fillna(df["total_beats"].median())

for c in [x for x in ["energy","danceability","valence","acousticness","speechiness","liveness", "instrumentalness"] if x in df.columns]:
    x = _num(df[c])
    df[f"{c}_squared"] = (x**2).clip(lower = 0)
    df[f"{c}_sqrt"] = np.sqrt(x.clip(lower = 0))
    engineered += [f"{c}_squared", f"{c}_sqrt"]

for c in core:
    df[f"{c}_pct"] = _num(df[c]).rank(pct = True).astype(float)
    engineered.append(f"{c}_pct")

for c in [x for x in ["energy","danceability","valence","tempo"] if x in df.columns]:
    try :
        q = pd.qcut(_num(df[c]), q = 5, duplicates = "drop")
        df[f"{c}_q"] = q.cat.codes.astype("int16")
        engineered.append(f"{c}_q")

        for code in sorted(df[f"{c}_q"].unique())[:4]:
            col = f"{c}_q{code}"
            df[col] = (df[f"{c}_q"] == code).astype("int8")
            engineered.append(col)
    except Exception:
        pass

if {"energy", "valence"}.issubset(df.columns):
    en, va = _num(df["energy"]), _num(df["valence"])
    df["edginess_index"] = (en * (1 - va)).clip(lower = 0, upper = 1).fillna(0)
    engineered.append("edginess_index")

need = {"energy", "danceability", "acouticness"}
if need.issubset(df.columns):
    en = _num(df["energy"])
    dance = _num(df["danceability"])
    acous = _num(df["acousticness"])
    df["chill_index"] = ((1 - en) * (1 - dance) * acous).clip(lower = 0, upper = 1).fillna(0)
    engineered.append("chill_index")

if {"speeciness", "instrumentalness"}.issubset(df.columns):
    speech = _num(df["speechiness"])
    inst = _num(df["instrumentalness"])
    df["speech_to_inst_ratio"] = speech / (inst + 1e-6)
    df["speech_to_inst_ratio"] = df["speech_to_inst_ratio"].replace([np.inf, -np.inf], np.nan).fillna(df["speech_to_inst_ratio"].median())
    engineered.append("speech_to_inst_ratio")

if {"acousticness", "energy"}.issubset(df.columns):
    acous = _num(df["acousticness"])
    en = _num(df["energy"])
    df["acoustic_to_energy_ratio"] = acous / (en + 1e-6)
    df["acoustic_to_energy_ratio"] = df["acoustic_to_energy_ratio"].replace([np.inf, -np.inf], np.nan).fillna(df["acoustic_to_energy_ratio"].median())
    engineered.append("acoustic_to_energy_ratio")

if "loudness" in df.columns:
    L = _num(df["loudness"])
    Lmin, Lmax = np.nanmin(L), np.nanmax(L)
    if pd.notna(Lmin) and pd.notna(Lmax) and Lmax > Lmin:
        df["loudness_norm"] = (L - Lmin) / (Lmax - Lmin)
        engineered.append("loudness_norm")

if {"mode", "valence"}.issubset(df.columns):
    df["mode_valence_combo_sq"] = _num(df["mode"]) * (_num(df["valence"]) ** 2)
    engineered.append("mode_valence_combo_sq")

fingerprint_cols = [c for c in [
    "acousticness","danceability","energy","liveness","loudness", "speechiness","tempo","valence","instrumentalness"
] if c in df.columns]

if len(fingerprint_cols) >= 3:
    _scaler = StandardScaler()
    Xf = _scaler.fit_transform(df[fingerprint_cols])
    pca = PCA(n_components = 3, random_state = 42)
    Z = pca.fit_transform(Xf)
    for i in range(Z.shape[1]):
        col = f"fp_pca_{i + 1}"
        df[col] = Z[:, i]
        engineered.append(col)

print(f"Engineered features created: {len(engineered)}")
print(", ".join(engineered[:12]) + (" ..." if len(engineered) > 12 else ""))

if "popularity" in df.columns:
    cols = [c for c in engineered if c in df.columns]
    corr_with_pop = (
        df[cols + ["popularity"]]
          .corr(numeric_only=True)["popularity"]
          .drop("popularity")
          .sort_values(ascending=False)
          .to_frame("corr_with_popularity")
          .round(3)
    )
    display(corr_with_pop.head(15))

Engineered features created: 51
energy_squared, energy_sqrt, danceability_squared, danceability_sqrt, valence_squared, valence_sqrt, acousticness_squared, acousticness_sqrt, speechiness_squared, speechiness_sqrt, liveness_squared, liveness_sqrt ...


,corr_with_popularity
loudness_pct,0.262
loudness_norm,0.243
fp_pca_1,0.193
energy_sqrt,0.137
danceability_sqrt,0.135
danceability_squared,0.117
danceability_pct,0.117
danceability_q,0.110
energy_pct,0.107
energy_q,0.105


In [10]:
import warnings
warnings.filterwarnings("ignore", message = "overflow encountered", category = RuntimeWarning)

CORR_THRESHOLD = 0.92
APPLY_LOG1P = True

num_cols = df.select_dtypes(include = [np.number]).columns.tolist()
target_col = "popularity"

try:
    engineered_features_leak
except NameError:
    engineered_features_leak = []
leak_set = set(engineered_features_leak)

try:
    engineered_features
except NameError:
    engineered_features = []
try:
    engineered_features_safe
except NameError:
    engineered_features_safe = []

X_candidates = [c for c in num_cols if c not in {target_col} and c not in leak_set]

log_added = []
if APPLY_LOG1P:
    skew = df[X_candidates].skew(numeric_only=True)
    log_feats = {}
    for c in X_candidates:
        if (df[c].min() >= 0) and (skew.get(c, 0) > 1.2):
            newc = f"log1p_{c}"
            log_feats[newc] = np.log1p(df[c])
            log_added.append(newc)
    if log_feats:
        df = pd.concat([df, pd.DataFrame(log_feats)], axis=1)
        X_candidates += log_added

priority = {}
for c in X_candidates:
    priority[c] = 0

for c in engineered_features:
    if c in priority:
        priority[c] = max(priority[c], 1)
for c in engineered_features_safe:
    if c in priority:
        priority[c] = max(priority[c], 2)
for c in log_added:
    priority[c] = max(priority.get(c, 0), 1)

X_df = df[X_candidates].copy()
X_df = X_df.loc[:, ~X_df.columns.duplicated(keep='first')]
X_candidates = X_df.columns.tolist()
corr_abs = X_df.corr(numeric_only = True).abs()

corr_with_y = (
    df[X_candidates + [target_col]]
      .corr(numeric_only=True)[target_col]
      .drop(labels=[target_col])
      .abs()
      .fillna(0.0)
)

def _as_float(val):
    try:
        return float(val)
    except Exception:
        arr = np.asarray(val).ravel()
        return float(arr[0]) if arr.size else 0.0

to_drop = set()
cols = corr_abs.columns.tolist()
for i in range(len(cols)):
    for j in range(i + 1, len(cols)):
        c1, c2 = cols[i], cols[j]
        if c1 in to_drop or c2 in to_drop:
            continue
        r = corr_abs.iloc[i, j]
        if r > CORR_THRESHOLD:
            p1, p2 = priority.get(c1, 0), priority.get(c2, 0)
            if p1 != p2:
                drop = c1 if p1 < p2 else c2
            else:
                y1 = _as_float(corr_with_y.get(c1, 0.0))
                y2 = _as_float(corr_with_y.get(c2,0))
                if y1 != y2:
                    drop = c1 if y1 < y2 else c2
                else:
                    v1 = float(np.nanvar(df[c1].values))
                    v2 = float(np.nanvar(df[c2].values))
                    drop = c1 if v1 < v2 else c2
            to_drop.add(drop)

X_kept = [c for c in X_candidates if c not in to_drop]
keep_aux = [col for col in ["popular_flag"] if col in df.columns]
final_cols = X_kept + [target_col] + keep_aux
features_df = df[final_cols].copy()

print(f"Initial numeric candidates: {len(num_cols)}")
print(f"Leaky excluded: {len(leak_set)}")
print(f"After adding logs: {len(X_candidates)} candidates")
print(f"Dropped for high correlation (>{CORR_THRESHOLD}): {len(to_drop)}")
print(f"Final kept features (X): {len(X_kept)}")
print(f"Final table shape (with target & aux): {features_df.shape}")

corr_final = (
    features_df.drop(columns = [target_col] + keep_aux, errors = "ignore")
                .assign(**{target_col: features_df[target_col]})
                .corr(numeric_only = True)[target_col]
                .drop(labels = [target_col])
                .abs()
                .sort_values(ascending = False)
                .head(20)
                .to_frame("abs_corr_with_popularity")
                .round(3)
)
print("\nTop 20 |corr| with target (post-pruning):")
display(corr_final)

final_feature_names = X_kept
pruned_highcorr = sorted(list(to_drop))
log_transformed_added = log_added

Initial numeric candidates: 67
Leaky excluded: 0
After adding logs: 95 candidates
Dropped for high correlation (>0.92): 45
Final kept features (X): 50
Final table shape (with target & aux): (130326, 52)

Top 20 |corr| with target (post-pruning):


,abs_corr_with_popularity
log1p_popular_flag,0.278
loudness_pct,0.262
loudness_norm,0.243
instrumentalness_pct,0.212
log1p_instrumentalness_squared,0.210
fp_pca_1,0.193
energy_sqrt,0.137
danceability_sqrt,0.135
log1p_acoustic_to_energy_ratio,0.133
log1p_energy_q0,0.132


**Export**

In [12]:
try:
    OUT_CSV
except NameError:
    ROOT_FALLBACK = os.path.abspath(os.path.join(os.getcwd(), "..")) if ("EDA" in os.getcwd() or "scripts" in os.getcwd()) else os.getcwd()
    OUT_CSV = os.path.join(ROOT_FALLBACK, "data", "clean_data", "spotify_features.csv")

try:
    META_JSON
except NameError:
    ROOT_FALLBACK = os.path.abspath(os.path.join(os.getcwd(), "..")) if ("EDA" in os.getcwd() or "scripts" in os.getcwd()) else os.getcwd()
    META_JSON = os.path.join(ROOT_FALLBACK, "artifacts", "feature_meta.json")

os.makedirs(os.path.dirname(OUT_CSV), exist_ok=True)
os.makedirs(os.path.dirname(META_JSON), exist_ok=True)

# ---- Check that features_df exists ----
assert "features_df" in globals() and isinstance(features_df, pd.DataFrame) and not features_df.empty, \
    "features_df missing or empty. Make sure Cell 6 ran successfully."

# ------------------------------------------------------------------
# 1️⃣  Add identifying text columns for later recommendation use
# ------------------------------------------------------------------
id_cols = [c for c in ["artist_name", "track_id", "track_name"] if c in df.columns]

if id_cols:
    enriched_df = pd.concat([df[id_cols].reset_index(drop=True),
                             features_df.reset_index(drop=True)], axis=1)
else:
    enriched_df = features_df.copy()

# Two output files: numeric-only & enriched
OUT_CSV_NUM  = OUT_CSV
OUT_CSV_INFO = OUT_CSV.replace(".csv", "_with_info.csv")

features_df.to_csv(OUT_CSV_NUM, index=False)
enriched_df.to_csv(OUT_CSV_INFO, index=False)

# ------------------------------------------------------------------
# 2️⃣  Build metadata JSON
# ------------------------------------------------------------------
def _maybe(name, default):
    return globals()[name] if name in globals() else default

meta = {
    "created_at_utc": datetime.utcnow().isoformat(timespec="seconds") + "Z",
    "input_clean_csv": _maybe("RAW_CSV", None),
    "output_features_csv": OUT_CSV_NUM,
    "output_features_with_info_csv": OUT_CSV_INFO,
    "output_meta_json": META_JSON,

    "n_rows": int(features_df.shape[0]),
    "n_cols_total": int(features_df.shape[1]),
    "target": "popularity",
    "popular_flag_included": bool("popular_flag" in features_df.columns),

    # configuration values
    "corr_threshold": _maybe("CORR_THRESHOLD", None),
    "apply_log1p": _maybe("APPLY_LOG1P", None),

    # feature inventories
    "engineered": _maybe("engineered", []),
    "engineered_safe": _maybe("engineered_features_safe", []),
    "engineered_leak": _maybe("engineered_features_leak", []),

    # bookkeeping from pruning
    "final_feature_names": _maybe("final_feature_names", []),
    "pruned_highcorr": _maybe("pruned_highcorr", []),
    "log_transformed_added": _maybe("log_transformed_added", []),

    # dtypes snapshot
    "dtypes": {c: str(t) for c, t in features_df.dtypes.items()},
}

with open(META_JSON, "w") as f:
    json.dump(meta, f, indent=2)

# ------------------------------------------------------------------
# 3️⃣  Confirmation prints + preview
# ------------------------------------------------------------------
print("✅ Export complete")
print(f"• Numeric-only CSV: {OUT_CSV_NUM} ({features_df.shape})")
print(f"• With song info:   {OUT_CSV_INFO} ({enriched_df.shape})")
print(f"• Meta JSON:        {META_JSON}")

preview_cols = pd.Series(features_df.columns[:25])
display(preview_cols.to_frame("first_25_columns"))

# Tiny preview CSV for README screenshots
PREVIEW_CSV = OUT_CSV_NUM.replace(".csv", "_preview_head10.csv")
features_df.head(10).to_csv(PREVIEW_CSV, index=False)
print(f"• Preview CSV (first 10 rows): {PREVIEW_CSV}")

✅ Export complete
• Numeric-only CSV: C:\DS\spotify-MRS\data\clean_data\spotify_features.csv ((130326, 52))
• With song info:   C:\DS\spotify-MRS\data\clean_data\spotify_features_with_info.csv ((130326, 54))
• Meta JSON:        C:\DS\spotify-MRS\artifacts\feature_meta.json


C:\Users\hayden\AppData\Local\Temp\ipykernel_2696\2062776659.py:45: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "created_at_utc": datetime.utcnow().isoformat(timespec="seconds") + "Z",


,first_25_columns
0,duration_ms
1,key
2,mode
3,tempo
4,time_signature
5,total_beats
6,energy_sqrt
7,danceability_sqrt
8,valence_sqrt
9,acousticness_sqrt


• Preview CSV (first 10 rows): C:\DS\spotify-MRS\data\clean_data\spotify_features_preview_head10.csv
